In [1]:
from scipy.io import loadmat
import numpy as np
from numpy import *
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pickle
from control import lqr, ss
from control.matlab import lsim
import random
from IPython.display import display, clear_output

**The Tabular value-fucntion methods on the cart and pole system**

Consider the cart-and-pole system here depicted:
<div>
<img src="https://drive.google.com/uc?export=view&id=1Yu5zZ57OEEYZ678yWLaU7h-Wq3M9vV9E" width="200">
</div>
whose dynamics follows the following equations:

\begin{align}
  \ddot{\theta}&=\frac{g \sin(\theta)\,+\,\cos(\theta) \left[ \frac{-F\,-\,m_p\,l\,\dot{\theta}^2 \sin \left(\theta\right)}{m_c\,+\,m_p}\right]-\frac{\mu_p\dot{\theta}}{m_p\,l}}{l\,\left[\frac{4}{3}-\frac{m_p\,cos^2\left(\theta\right)}{m_c\,+\,m_p}\right]}
\end{align}
>>>>>>>>>>>>>>>>>>>>>>>>$(*)$

\begin{align}
  \ddot{p}_c&=\frac{F\,+\,m_p\,l\,\left[\dot{\theta}^2\,\sin\left(\theta\right)\,-\,\ddot{\theta}\,\cos(\theta)\right]}{m_c\,+\,m_p}
\end{align}

where $g = 9.8\,\text{m/sec}$ is the gravitational acceleration, $m_c = 1\,\text{kg}$ is the cart mass, $m_p = 0.1\,\text{kg}$ is the pole mass, $l = 0.5\,\text{m}$ is the half-pole length, $\mu_p = 0.000002$ is the pole on cart friction coefficient.

1. Given the $\mathcal{X}$, $\mathcal{U}$ of the first hands-on define: the set of possible initial conditions of an episode; the terminal conditions of the episode


The set of all possibile initial conditions is:

$$
x = \left( \begin{array}{ccc}
                \theta \\
                \dot \theta \\
                p_c \\
                \dot p_c
\end{array} \right)
=
\left( \begin{array}{ccc}
                  x_1 \\
                  x_2 \\
                  x_3 \\
                  x_4
\end{array} \right)
$$

And the terminal episodes are:
1. if the pole felt, $\theta = \{-\frac{\pi}{4}; \frac{\pi}{4}\} rad$
2. if the cart is going out of the constraints $|p_c| > 5m$
3. if the control input force is too high $|u| > 10N$
4. If the episode is taking more than $10s$ to complete (500 time steps)

2. Provide a proper discretization of the state space $\mathcal{X}$, and define the size of the action-value function $Q$




3. Create a code able to select the discretized version of a measured state $x$.


4.    Given the terminal conditions of the episode (selected in 1.) create a code able to perform a fixed number of episodes of a fixed number of steps, starting from random initial conditions and applying random inputs.

5.   Create a function able to perform $\epsilon$-greedy policy for a chosen $\epsilon$.

6. Create a code that applies Tabular Q-Learning algorithm.

The possible inputs are either $F=10$ or $F=-10$

1. discretizza in piu range possibili
2. 

In [2]:
def cart_and_pole_odeint(x, t, F):
  dxdt = np.zeros_like(x)

  dxdt[0] = x[1] #tetap
  dxdt[1] = (g*sin(x[0])+ cos(x[0])*((-F-m*l*(x[1]**2)*sin(x[0]))/(mc+m))-((miup*x[1])/(m*l))) / (l*((4/3)-((m*(cos(x[0])**2))/(mc+m))))    #teta2p
  dxdt[2] = x[3] #xp
  dxdt[3] = (F+m*l*((x[1]**2)*sin(x[0])-dxdt[1]*cos(x[0])))/(mc+m) #x2p
  return dxdt

g = 9.8 # gravitational acceleration
mc = 1  # cart mass [kg]
l = 0.5 # half-pole length [m]
m = 0.1 # pole mass [kg]
miup = 2e-6 # pole friction coefficient

time_step=0.02
t0_odeint=0
x0_odeint = np.array([np.deg2rad(1), 0.0, 0.0, 0.0]) # start out of equilibrium
X_odeint=[x0_odeint]

In [3]:
lowerBounds = [
    deg2rad(-25),
    -1,
    -5,
    -2
]
upperBounds = [- i for i in lowerBounds]
numberOfBins = [
    10,
    10,
    10,
    10
]

def discretize_state(state):
    angle    =      state[0]
    angularVelocity=state[1]
    position =      state[2]
    velocity =      state[3]

    poleAngleBin=np.linspace(lowerBounds[0],upperBounds[0],numberOfBins[0])
    poleAngleVelocityBin=np.linspace(lowerBounds[1],upperBounds[1],numberOfBins[1])
    cartPositionBin=np.linspace(lowerBounds[2],upperBounds[2],numberOfBins[2])
    cartVelocityBin=np.linspace(lowerBounds[3],upperBounds[3],numberOfBins[3])

    indexAngle=np.maximum(np.digitize(angle,poleAngleBin)-1,0)
    indexAngularVelocity=np.maximum(np.digitize(angularVelocity,poleAngleVelocityBin)-1,0)
    indexPosition=np.maximum(np.digitize(position,cartPositionBin)-1,0)
    indexVelocity=np.maximum(np.digitize(velocity,cartVelocityBin)-1,0)

    return tuple([indexAngle,indexAngularVelocity,indexPosition,indexVelocity])

def is_terminal_state(dstate):
    # discretized state
    if dstate[0] == 0 or dstate[0] == numberOfBins[0]-1:
        return True
    if dstate[2] == 0 or dstate[2] == numberOfBins[2]-1:
        return True
    return False

In [4]:
total_episodes = 1000
max_steps= 500
learning_rate = 0.01
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.001             # Exponential decay rate for exploration prob

action_size = 2 # left or right

In [5]:
qtable = np.zeros(tuple(numberOfBins) + (action_size, ))

In [6]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    clear_output(wait=True)
    display('Episode '+str(episode))

    # Reset the environment
    state_c = x0_odeint
    state = discretize_state(state_c)
    step = 0
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state])

        # Else doing a random choice --> exploration
        else:
            action = random.randint(0,1)
            
        # convert action into force
        if action == 1:
            F = 10
        else:
            F = -10

        # Simulate the system
        # new_state, reward, done, info, _ = env.step(action)
        t1_odeint = t0_odeint+0.02
        t_odeint = np.array([t0_odeint, t1_odeint])
        x_odeint = odeint(cart_and_pole_odeint, state_c, t_odeint, args=(F, ))
        new_state_c = x_odeint[1]
        new_state = discretize_state(new_state_c)
        
        if is_terminal_state(new_state):
            reward = -100
        else:
            reward = 1

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state + (action,)] = qtable[state + (action,)] + learning_rate * (reward + gamma * np.max(qtable[new_state]) - qtable[state + (action,)])
        
        total_rewards =total_rewards + reward
        
        # Our new state is state
        state = new_state
        state_c = new_state_c

        
        # If done (if we're dead) : finish episode
        if is_terminal_state(new_state):
            break
        
    episode += 1
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
#print(qtable)
#print(epsilon)


'Episode 12'

KeyboardInterrupt: 

In [ ]:
plt.plot(rewards, 'r', label='U(t)')

### Run the simulation

In [ ]:
X_odeint = [x0_odeint]
uvec = []

state_c = x0_odeint
state = discretize_state(state_c)
total_rewards = 0

for step in range(max_steps):
    action = np.argmax(qtable[state])

    # convert action into force
    if action == 1:
        F = 10
    else:
        F = -10
        
    uvec.append(F)

    # Simulate the system
    # new_state, reward, done, info, _ = env.step(action)
    t1_odeint = t0_odeint+0.02
    t_odeint = np.array([t0_odeint, t1_odeint])
    x_odeint = odeint(cart_and_pole_odeint, state_c, t_odeint, args=(F, ))
    new_state_c = x_odeint[1]
    new_state = discretize_state(new_state_c)
    X_odeint=np.append(X_odeint, [state_c], axis= 0)

    if is_terminal_state(new_state):
        reward = -100
    else:
        reward = 1

    # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
    # qtable[new_state,:] : all the actions we can take from new state
    # qtable[state + (action,)] = qtable[state + (action,)] + learning_rate * (reward + gamma * np.max(qtable[new_state]) - qtable[state + (action,)])

    total_rewards =total_rewards + reward

    # Our new state is state
    state = new_state
    state_c = new_state_c

    # If done (if we're dead) : finish episode
    if is_terminal_state(new_state):
        break

In [ ]:
print("Time:", np.arange(0.0, 10, time_step)[:len(uvec)][-1])
time_sim=np.arange(0.0, 10+time_step, time_step)

plt.rcParams['figure.figsize'] = [10, 15]
fig, axs = plt.subplots(5)
fig.subplots_adjust(hspace=0.5)

axs[0].plot(np.arange(0.0, 10, time_step)[:len(uvec)], uvec, 'r', label='U(t)')
axs[0].set_title("U")

axs[1].plot(time_sim[:X_odeint[:,0].shape[0]], X_odeint[:,0], 'b')
axs[1].set_title("theta(t)")

axs[2].plot(time_sim[:X_odeint[:,1].shape[0]], X_odeint[:,1], 'b')
axs[2].set_title("dot theta(t)")

axs[3].plot(time_sim[:X_odeint[:,2].shape[0]], X_odeint[:,2], 'b')
axs[3].set_title("p_c(t)")

axs[4].plot(time_sim[:X_odeint[:,3].shape[0]], X_odeint[:,3], 'b')
axs[4].set_title("dot p_c(t)")

7.    Copy and paste your Q-learning algorithm and apply the changes needed to convert it into a SARSA algorithm

In [ ]:
qtable_sarsa = np.zeros(tuple(numberOfBins) + (action_size, ))

In [ ]:
total_episodes = 1000

In [ ]:
rewards = []

In [ ]:
# List of rewards
desired_state_c = [0,0,0,0]
desired_state = discretize_state(desired_state_c)

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    clear_output(wait=True)
    display('Episode '+str(episode))

    # Reset the environment
    state_c = x0_odeint
    state = discretize_state(state_c)
    step = 0
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable_sarsa[state])

        # Else doing a random choice --> exploration
        else:
            action = random.randint(0,1)
            
        # convert action into force
        if action == 1:
            F = 10
        else:
            F = -10

        # Simulate the system
        t1_odeint = t0_odeint+0.02
        t_odeint = np.array([t0_odeint, t1_odeint])
        x_odeint = odeint(cart_and_pole_odeint, state_c, t_odeint, args=(F, ))
        new_state_c = x_odeint[1]
        new_state = discretize_state(new_state_c)
        
        if is_terminal_state(new_state):
            reward = -100
        else:
            reward = 1

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable_sarsa[state + (action,)] = qtable_sarsa[state + (action,)] + learning_rate * (reward + gamma * qtable_sarsa[new_state + (action,)] - qtable_sarsa[desired_state + (action,)])
        
        total_rewards =total_rewards + reward
        
        # Our new state is state
        state = new_state
        state_c = new_state_c

        
        # If done (if we're dead) : finish episode
        if is_terminal_state(new_state):
            break
        
    episode += 1
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
#print(qtable)
#print(epsilon)


In [ ]:
plt.plot(rewards, 'r', label='U(t)')

## Export the q matrix

In [ ]:
with open("qtable.npy", "wb") as f:
    np.save(f, qtable)

In [ ]:
with open("qtable_sarsa.npy", "wb") as f:
    np.save(f, qtable_sarsa)

### Run the simulation

In [ ]:
X_odeint = [x0_odeint]
uvec = []

state_c = x0_odeint
state = discretize_state(state_c)
total_rewards = 0

for step in range(max_steps):
    action = np.argmax(qtable_sarsa[state])

    # convert action into force
    if action == 1:
        F = 10
    else:
        F = -10
        
    uvec.append(F)

    # Simulate the system
    # new_state, reward, done, info, _ = env.step(action)
    t1_odeint = t0_odeint+0.02
    t_odeint = np.array([t0_odeint, t1_odeint])
    x_odeint = odeint(cart_and_pole_odeint, state_c, t_odeint, args=(F, ))
    new_state_c = x_odeint[1]
    new_state = discretize_state(new_state_c)
    X_odeint=np.append(X_odeint, [state_c], axis= 0)

    if is_terminal_state(new_state):
        reward = -100
    else:
        reward = 1

    total_rewards =total_rewards + reward

    # Our new state is state
    state = new_state
    state_c = new_state_c

    # If done (if we're dead) : finish episode
    if is_terminal_state(new_state):
        break

In [ ]:
print("Time:", np.arange(0.0, 10, time_step)[:len(uvec)][-1])
time_sim=np.arange(0.0, 10+time_step, time_step)

plt.rcParams['figure.figsize'] = [10, 15]
fig, axs = plt.subplots(5)
fig.subplots_adjust(hspace=0.5)

axs[0].plot(np.arange(0.0, 10, time_step)[:len(uvec)], uvec, 'r', label='U(t)')
axs[0].set_title("U")

axs[1].plot(time_sim[:X_odeint[:,0].shape[0]], X_odeint[:,0], 'b')
axs[1].set_title("theta(t)")

axs[2].plot(time_sim[:X_odeint[:,1].shape[0]], X_odeint[:,1], 'b')
axs[2].set_title("dot theta(t)")

axs[3].plot(time_sim[:X_odeint[:,2].shape[0]], X_odeint[:,2], 'b')
axs[3].set_title("p_c(t)")

axs[4].plot(time_sim[:X_odeint[:,3].shape[0]], X_odeint[:,3], 'b')
axs[4].set_title("dot p_c(t)")